# Graph Theory Fundamentals - Exercises

This notebook contains exercises for Section 2.4: Graph Theory Fundamentals from the Applied Mathematics for Complex Systems Modeling course.

## Topics Covered
- Graphs, Vertices, and Edges
- Paths, Cycles, and Connectivity
- Graph Representations: Adjacency Matrices and Lists
- Graph Properties and Measures
- Special Graph Types and Structures
- Applications to Network Analysis

Let's begin by importing the necessary libraries for our exercises.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import seaborn as sns

# Set plot styling
plt.style.use('ggplot')
sns.set_context("notebook", font_scale=1.5)

# Check versions
print(f"NetworkX version: {nx.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.__version__}")

## Solved Exercise: Social Network Analysis

In this exercise, we'll analyze a small social network using graph theory concepts. We'll create a graph representing friendships between individuals, then analyze its properties and visualize it.

### Problem Statement

Consider a small group of 8 people with the following friendship connections:
- Alice is friends with Bob, Charlie, and Diana
- Bob is friends with Alice, Charlie, and Eva
- Charlie is friends with Alice, Bob, and Frank
- Diana is friends with Alice and Grace
- Eva is friends with Bob and Frank
- Frank is friends with Charlie, Eva, and Hector
- Grace is friends with Diana and Hector
- Hector is friends with Frank and Grace

Tasks:
1. Represent this social network as a graph using NetworkX
2. Visualize the graph
3. Calculate basic graph properties (degree distribution, clustering coefficient, diameter)
4. Identify the most central individual(s) using different centrality measures
5. Find all possible paths between Alice and Hector, and identify the shortest path

Let's solve this step by step.

### Step 1: Create the graph representation

First, we'll create the graph by adding nodes (people) and edges (friendships).

In [ ]:
# Create an undirected graph
G = nx.Graph()

# Add nodes (people)
people = ["Alice", "Bob", "Charlie", "Diana", "Eva", "Frank", "Grace", "Hector"]
G.add_nodes_from(people)

# Add edges (friendships)
friendships = [
    ("Alice", "Bob"), ("Alice", "Charlie"), ("Alice", "Diana"),
    ("Bob", "Charlie"), ("Bob", "Eva"),
    ("Charlie", "Frank"),
    ("Diana", "Grace"),
    ("Eva", "Frank"),
    ("Frank", "Hector"),
    ("Grace", "Hector")
]
G.add_edges_from(friendships)

# Print basic information about the graph
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")
print(f"\nNodes: {G.nodes()}")
print(f"\nEdges: {G.edges()}")

### Step 2: Visualize the social network graph

Now let's create a visualization of our social network to better understand the relationships.

In [ ]:
# Create a function for visualization with customizable node colors and sizes
def visualize_graph(G, node_size=300, node_color='skyblue', title="Graph Visualization"):
    plt.figure(figsize=(10, 8))
    
    # Use a spring layout for positioning
    pos = nx.spring_layout(G, seed=42)  # Seed for reproducibility
    
    # Draw the graph
    nx.draw_networkx(
        G, pos, 
        node_size=node_size,
        node_color=node_color,
        edge_color='gray',
        font_size=12,
        width=2,
        alpha=0.8
    )
    
    plt.title(title, fontsize=16)
    plt.axis('off')  # Turn off the axis
    plt.tight_layout()
    plt.show()

# Visualize our social network
visualize_graph(G, title="Social Network of 8 Friends")

### Step 3: Calculate basic graph properties

Let's calculate some basic properties of our social network graph.

In [ ]:
# 1. Degree distribution - how many connections each person has
degrees = dict(G.degree())
print("Degree of each person (number of friends):")
for person, degree in sorted(degrees.items()):
    print(f"{person}: {degree} friends")

# Create a bar plot of the degree distribution
plt.figure(figsize=(10, 6))
plt.bar(degrees.keys(), degrees.values(), color='skyblue')
plt.title('Degree Distribution: Number of Friends per Person', fontsize=14)
plt.xlabel('Person', fontsize=12)
plt.ylabel('Number of Friends', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# 2. Clustering coefficient - measure of the degree to which nodes tend to cluster together
clustering = nx.clustering(G)
avg_clustering = nx.average_clustering(G)

print("\nClustering coefficient for each person:")
for person, coef in sorted(clustering.items()):
    print(f"{person}: {coef:.3f}")
print(f"\nAverage clustering coefficient: {avg_clustering:.3f}")

# 3. Calculate the diameter (the maximum shortest path length)
if nx.is_connected(G):
    diameter = nx.diameter(G)
    print(f"\nDiameter of the network: {diameter}")
else:
    print("\nThe graph is not connected, so diameter is undefined.")
    
# 4. Calculate the average shortest path length
if nx.is_connected(G):
    avg_path_length = nx.average_shortest_path_length(G)
    print(f"Average shortest path length: {avg_path_length:.3f}")
else:
    print("The graph is not connected, so average shortest path length is undefined.")

### Step 4: Identify the most central individuals

Now we'll calculate different centrality measures to determine who is most central to this social network.

Different centrality measures capture different aspects of importance in a network:
- **Degree centrality**: Number of direct connections
- **Betweenness centrality**: How often a node lies on shortest paths between other nodes
- **Closeness centrality**: How close a node is to all other nodes
- **Eigenvector centrality**: How connected a node is to other well-connected nodes

In [ ]:
# Calculate centrality measures
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G)

# Create a DataFrame to display all centrality measures together
centrality_df = pd.DataFrame({
    'Degree': degree_centrality,
    'Betweenness': betweenness_centrality,
    'Closeness': closeness_centrality,
    'Eigenvector': eigenvector_centrality
})

# Sort by betweenness centrality (often a good overall measure)
centrality_df = centrality_df.sort_values('Betweenness', ascending=False)

# Display the table
print("Centrality measures for each person (sorted by betweenness):")
print(centrality_df.round(3))

# Create visualizations of the centrality measures
plt.figure(figsize=(14, 10))

# Create subplots for each centrality measure
measures = ['Degree', 'Betweenness', 'Closeness', 'Eigenvector']
for i, measure in enumerate(measures):
    plt.subplot(2, 2, i+1)
    
    # Create a color map for nodes based on centrality
    centrality_values = list(centrality_df[measure])
    
    # Spring layout for consistent positioning across plots
    pos = nx.spring_layout(G, seed=42)
    
    # Draw the graph with node sizes proportional to centrality
    node_sizes = [v * 1500 + 300 for v in centrality_values]  # Scale for visibility
    nx.draw_networkx(
        G, pos,
        node_size=node_sizes,
        node_color=centrality_values,
        cmap=plt.cm.viridis,
        edge_color='gray',
        font_size=10,
        width=1.5,
        alpha=0.8
    )
    
    plt.title(f"{measure} Centrality", fontsize=14)
    plt.axis('off')

plt.tight_layout()
plt.show()

### Step 5: Find all paths between Alice and Hector

Finally, let's find all possible paths between Alice and Hector, and identify the shortest path.

In [ ]:
# Find all simple paths between Alice and Hector
all_paths = list(nx.all_simple_paths(G, source="Alice", target="Hector"))

print(f"Number of distinct paths between Alice and Hector: {len(all_paths)}")
print("\nAll paths from Alice to Hector:")
for i, path in enumerate(all_paths, 1):
    print(f"Path {i} (length {len(path)-1}): {' -> '.join(path)}")

# Find the shortest path(s)
shortest_path = nx.shortest_path(G, source="Alice", target="Hector")
shortest_length = len(shortest_path) - 1

print(f"\nShortest path (length {shortest_length}): {' -> '.join(shortest_path)}")

# Visualize the shortest path
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, seed=42)

# Draw the full graph with reduced opacity
nx.draw_networkx(
    G, pos,
    node_size=300,
    node_color='lightgray',
    edge_color='lightgray',
    font_size=10,
    width=1.5,
    alpha=0.5
)

# Create a subgraph with just the shortest path
path_edges = list(zip(shortest_path[:-1], shortest_path[1:]))
path_nodes = shortest_path

# Draw the path edges and nodes with higher opacity and different colors
nx.draw_networkx_nodes(
    G, pos,
    nodelist=path_nodes,
    node_size=500,
    node_color='crimson',
    alpha=1.0
)
nx.draw_networkx_edges(
    G, pos,
    edgelist=path_edges,
    width=3,
    edge_color='crimson',
    alpha=1.0
)
nx.draw_networkx_labels(
    G, pos,
    labels={node: node for node in path_nodes},
    font_size=12,
    font_weight='bold'
)

plt.title(f"Shortest Path from Alice to Hector ({shortest_length} steps)", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

### Summary of Findings

From our analysis of this social network, we can conclude that:

1. The network has 8 people (nodes) and 10 friendship connections (edges).
2. Some people have more friends than others, with the degree ranging from 2 to 3 connections.
3. The clustering coefficient varies among individuals, indicating different levels of social cohesion.
4. The network has a diameter of 3, meaning that any two people can reach each other through at most 3 connections.
5. Different centrality measures highlight different important people in the network:
   - By degree centrality: People with the most direct friends
   - By betweenness centrality: People who serve as bridges between different groups
   - By closeness centrality: People who can reach others most efficiently
   - By eigenvector centrality: People connected to other well-connected people
6. There are multiple paths between Alice and Hector, with the shortest path having a length of 3.

This demonstrates how graph theory provides powerful tools for analyzing social networks and understanding the relationships and structures within them.

## Unsolved Exercise: Transportation Network Analysis

In this exercise, you'll analyze a transportation network for a small city, applying graph theory concepts to identify efficient routes and critical infrastructure.

### Problem Statement

Consider a city with 10 districts connected by roads. The travel time (in minutes) between connected districts is provided in the following table:

| From/To | A | B | C | D | E | F | G | H | I | J |
|---------|---|---|---|---|---|---|---|---|---|---|
| A       | 0 | 5 | 0 | 0 | 7 | 0 | 0 | 0 | 0 | 0 |
| B       | 5 | 0 | 8 | 0 | 0 | 10| 0 | 0 | 0 | 0 |
| C       | 0 | 8 | 0 | 6 | 0 | 0 | 12| 0 | 0 | 0 |
| D       | 0 | 0 | 6 | 0 | 0 | 0 | 0 | 5 | 0 | 0 |
| E       | 7 | 0 | 0 | 0 | 0 | 3 | 0 | 0 | 9 | 0 |
| F       | 0 | 10| 0 | 0 | 3 | 0 | 4 | 0 | 0 | 0 |
| G       | 0 | 0 | 12| 0 | 0 | 4 | 0 | 7 | 0 | 8 |
| H       | 0 | 0 | 0 | 5 | 0 | 0 | 7 | 0 | 0 | 11|
| I       | 0 | 0 | 0 | 0 | 9 | 0 | 0 | 0 | 0 | 6 |
| J       | 0 | 0 | 0 | 0 | 0 | 0 | 8 | 11| 6 | 0 |

A value of 0 means there is no direct road connecting the districts.

### Tasks

Complete the following tasks:

1. Create a weighted directed graph representation of this transportation network
2. Visualize the network with edge weights (travel times) displayed
3. Find the shortest path (minimum travel time) from district A to district J
4. Identify the most central district(s) using appropriate centrality measures
5. If one road needs to be closed for maintenance, which road would cause the greatest increase in average travel time across the network? (Hint: consider edge betweenness centrality)
6. Suggest 1-2 new road connections that would most improve the network's efficiency

### Getting Started

Here's some code to help you get started:

In [ ]:
# Create a directed graph with weighted edges
city_network = nx.DiGraph()

# Add nodes (districts)
districts = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
city_network.add_nodes_from(districts)

# Edge weights (travel times) matrix
travel_times = np.array([
    [0, 5, 0, 0, 7, 0, 0, 0, 0, 0],
    [5, 0, 8, 0, 0, 10, 0, 0, 0, 0],
    [0, 8, 0, 6, 0, 0, 12, 0, 0, 0],
    [0, 0, 6, 0, 0, 0, 0, 5, 0, 0],
    [7, 0, 0, 0, 0, 3, 0, 0, 9, 0],
    [0, 10, 0, 0, 3, 0, 4, 0, 0, 0],
    [0, 0, 12, 0, 0, 4, 0, 7, 0, 8],
    [0, 0, 0, 5, 0, 0, 7, 0, 0, 11],
    [0, 0, 0, 0, 9, 0, 0, 0, 0, 6],
    [0, 0, 0, 0, 0, 0, 8, 11, 6, 0]
])

# Add weighted edges based on the matrix
for i in range(len(districts)):
    for j in range(len(districts)):
        if travel_times[i, j] > 0:  # If there's a direct connection
            city_network.add_edge(districts[i], districts[j], weight=travel_times[i, j])

# Verify the graph
print(f"Number of nodes: {city_network.number_of_nodes()}")
print(f"Number of edges: {city_network.number_of_edges()}")

# Your analysis code here...
# ...
# ...


### Hints for Solving the Problem

Here are some functions that you might find helpful:

1. For finding the shortest path:
   - `nx.dijkstra_path(G, source, target, weight='weight')`
   - `nx.dijkstra_path_length(G, source, target, weight='weight')`

2. For centrality measures in weighted networks:
   - `nx.degree_centrality(G)`
   - `nx.betweenness_centrality(G, weight='weight')`
   - `nx.closeness_centrality(G, distance='weight')`
   - `nx.load_centrality(G, weight='weight')`

3. For visualizing weighted graphs:
   - You can use `nx.draw_networkx_edge_labels()` to display the weights

4. For edge betweenness centrality:
   - `nx.edge_betweenness_centrality(G, weight='weight')`

5. To analyze the impact of removing an edge:
   - Create a copy of the graph: `H = G.copy()`
   - Remove an edge: `H.remove_edge(u, v)`
   - Calculate average shortest path length: `nx.average_shortest_path_length(H, weight='weight')`
   
Remember to handle cases where removing an edge might disconnect the graph.

Good luck!